In [109]:
#!pip install haversine
#!pip install shapely
#!pip install geopandas
#!pip install geojson
#!pip install folium
#!pip install tqdm

In [110]:
import requests
import random
import pandas as pd 
from tqdm import tqdm
#from cred_here import YOUR_API_KEY
import json 

import ast

# Tools

import folium
from shapely.geometry import Polygon
import numpy as np
import geojson
import folium
import matplotlib.pyplot as plt
import geopandas as gpd
from tqdm import tqdm
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import shapely.wkt
from haversine import haversine, Unit
import random
import time
from pyproj import Geod

from shapely import wkt
from geopandas import datasets, GeoDataFrame, read_file, points_from_xy
from geopandas.tools import overlay
from geopandas.tools import sjoin

from folium.plugins import MeasureControl
from folium.plugins import MarkerCluster

from tqdm import tqdm
import json

import requests
import random
import time 

In [111]:
def deg_to_dec(degrees, minutes, seconds, pole):
    
    if pole == 'S' or pole == 'W':
        sign = -1
    else:
        sign = 1

    decimal = sign * (abs(degrees) + (minutes / 60.0) + (seconds / 3600.0))

    return decimal

def convertir (coordenada,tipo):
        
    coordenada = coordenada.replace("(","").replace(")","").replace("\'","").replace("°","").replace("'","").replace('"','').split(',')
    coord1 = coordenada[0].replace("'","").split(" ")
    valor = None
    if 'NA' not in coord1:
        if tipo == 'Latitud':
            valor = deg_to_dec(float(coord1[0]), float(coord1[1]), float(coord1[2]), coord1[3])
        else :
            valor = deg_to_dec(float(coord1[0]), float(coord1[1]), float(coord1[2]), coord1[3])
        return float(valor)
    else :
        valor = 'Empty'
        return valor

In [112]:
df_map = pd.read_csv('DATASET_FINAL.csv')

In [113]:
df_map.head(2)

,Cod.Departamento,Departamento,Cod.Municipio,Municipio,Agente,Bandera,Direccion,Producto,Precio,Estado,Registro,Periodo,Mes,Direccion_2,Full_Address,Coords
0,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,BIODIESEL EXTRA,8400,1,05/02/2018 12:00:00 AM,2018,5,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia","(""4° 15' 0.00000'' N"", ""74° 10' 59.98800'' W"")"
1,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,GASOLINA CORRIENTE OXIGENADA,9095,1,05/02/2018 12:00:00 AM,2018,5,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia","(""4° 15' 0.00000'' N"", ""74° 10' 59.98800'' W"")"


In [114]:
df_map = df_map.drop_duplicates(subset=['Agente','Bandera','Producto'])

In [115]:
len(df_map)

1641

In [116]:
df_map['Coords'].info()

<class 'pandas.core.series.Series'>
Int64Index: 1641 entries, 0 to 1640
Series name: Coords
Non-Null Count  Dtype 
--------------  ----- 
1641 non-null   object
dtypes: object(1)
memory usage: 25.6+ KB


In [117]:
len(df_map[df_map['Coords'].str.contains("'NA'")])

3

In [118]:
df_map.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1641 entries, 0 to 1640
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Cod.Departamento  1641 non-null   int64 
 1   Departamento      1641 non-null   object
 2   Cod.Municipio     1641 non-null   int64 
 3   Municipio         1641 non-null   object
 4   Agente            1641 non-null   object
 5   Bandera           1641 non-null   object
 6   Direccion         1641 non-null   object
 7   Producto          1641 non-null   object
 8   Precio            1641 non-null   int64 
 9   Estado            1641 non-null   int64 
 10  Registro          1641 non-null   object
 11  Periodo           1641 non-null   int64 
 12  Mes               1641 non-null   int64 
 13  Direccion_2       1641 non-null   object
 14  Full_Address      1641 non-null   object
 15  Coords            1641 non-null   object
dtypes: int64(6), object(10)
memory usage: 217.9+ KB


In [119]:
df_map['Coords'][0].replace('(','').replace(')','').replace("\'","").replace("°","").replace("'","").replace('"','').split(',')
#df_map['Coords'][2]

['4 15 0.00000 N', ' 74 10 59.98800 W']

In [120]:
df_map['Coords'][2].replace("(","").replace("\'","").replace("°","").replace("'","").replace('"','').split(',')[0]

'4 15 0.00000 N'

In [121]:
df_map['Coords'][2].replace(")","").replace("\'","").replace("°","").replace("'","").replace('"','').split(',')[1].strip(" ")

'74 10 59.98800 W'

In [122]:
df_map['Latitud'] = df_map['Coords'].apply(lambda x : convertir(x,'Latitud') if x != "('NA','NA')" else "Empty")
df_map['Longitud'] = df_map['Coords'].apply(lambda x : convertir(x,'Longitud') if x != "('NA','NA')" else "Empty")

#df_map['LAT'] = df_map['Coords'].apply(lambda x : float(x.replace('(','').replace(')','').split(',')[0]) if x !=  "('NA', 'NA')"  else 'Empty')
#df_map['LNG'] = df_map['Coords'].apply(lambda x : float(x.replace('(','').replace(')','').split(',')[1]) if x !=  "('NA', 'NA')"  else 'Empty')

In [123]:
df_map['Latitud'].value_counts()

4.250000    785
6.251840    162
4.605343    151
4.626140     47
6.337320     44
           ... 
6.336765      2
6.289300      1
6.208349      1
6.274470      1
6.227714      1
Name: Latitud, Length: 102, dtype: int64

In [124]:
df_map['Longitud'].value_counts()

4.250000    785
6.251840    162
4.605343    151
4.626140     47
6.337320     44
           ... 
6.336765      2
6.289300      1
6.208349      1
6.274470      1
6.227714      1
Name: Longitud, Length: 102, dtype: int64

# Funciones a usar 

In [125]:
def GetLatLon2(Address,YOUR_API_KEY): 

    url2_geocode  = f'https://geocode.search.hereapi.com/v1/geocode?q={Address}&apiKey='+YOUR_API_KEY
    
    try:
        response = requests.get(url2_geocode).json()
        CleanAddress = response['items'][0]['title'].upper()
        LAT = response['items'][0]['position']['lat']
        LON = response['items'][0]['position']['lng']
        results = [CleanAddress,round(LAT,7),round(LON,7)]
    except:
        results = ['NotFound','NA','NA']
    return results

def GetLatLon2_google(Address,YOUR_API_KEY): 

    api_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={Address}&key={YOUR_API_KEY}'
    try:
        j = requests.get(api_url).json()
        CleanAddress = str(j['results'][0]['formatted_address']).upper()
        LAT = j['results'][0]['geometry']['location']['lat']
        LON = j['results'][0]['geometry']['location']['lng']
        results = [CleanAddress,round(LAT,7),round(LON,7)]
    except:
        results = ['NotFound','NA','NA']
    return results,j 

def GetLatLon2__opencage(Address,YOUR_API_KEY):
    
    url=f'https://api.opencagedata.com/geocode/v1/geojson?q={Address}&key={YOUR_API_KEY}&pretty=1'

    try : 
        
        response = requests.get(url).json()
        #CleanAddress = response['items'][0]['title'].upper()
        CleanAddress = Address
        LAT = response['features'][0]['properties']['annotations']['DMS']['lat']
        LNG = response['features'][0]['properties']['annotations']['DMS']['lng']
        
        results = [CleanAddress,convertir(LAT,'Latitud'),convertir(LNG,'Logitud')]
        
    except :
        
        results = ['Not Found','NA','NA']
        
    return results 



def cal_dist(geo_source,point2,unit):
    
    
    if unit == 'Km':
        distance = haversine(geo_source, point2,Unit.KILOMETERS)
    elif unit == 'm':
        distance = haversine(geo_source, point2,Unit.METERS)
    elif unit == 'miles':
        distance = haversine(geo_source, point2,Unit.MILES)
    
    return round(distance,2)


def distance_estac(geo_source,df,radio,unit):
    
    
    distancia = []
    source = []


    for i in tqdm(range(len(df)),colour = 'green'):
        distancia.append(cal_dist(geo_source,df['POINT'][i],unit))
        source.append(geo_source)

    new_df = df.copy()
    new_df['SOURCE'] = source
    new_df['DISTANCE'] = distancia
    new_df = new_df[new_df['DISTANCE']>=radio]
    new_df = new_df.reset_index(drop=True)
    #new_df = new_df.drop(columns ='index')
    return new_df.sort_values(by='DISTANCE',ascending=True)

def transform_df_map(df):
    

    coordenadas = []

    for i in range(len(df)):
        
        try :
            
            coord = float(df['Latitud'][i]),float(df['Longitud'][i])
            coordenadas.append(coord)

        except :
            
            coordenadas.append('EMPTY')
            
    df['POINT'] = coordenadas
    df = df[df['POINT']!='EMPTY']
    df = df.reset_index(drop=True)
    #df = df.drop(columns = 'index')
    new_df = df.copy()
    
    return new_df

# Paso a Paso de la App

1. Filtrar la ciudad
2. Ingresar la central location
3. Configurar el radio y la unidad de medida
4. Calcular la distancia


# Filtrar ciudad

In [126]:
cities = list(df_map['Municipio'].unique())
cities

['BOGOTA D.C.', 'MEDELLIN', 'BELLO']

In [127]:
df_city = df_map[df_map['Municipio']==cities[0]]
df_city.reset_index(drop = True,inplace =  True)
#df_city.drop(columns= 'index' , inplace = True)

In [128]:
df_city.head(2)

,Cod.Departamento,Departamento,Cod.Municipio,Municipio,Agente,Bandera,Direccion,Producto,Precio,Estado,Registro,Periodo,Mes,Direccion_2,Full_Address,Coords,Latitud,Longitud
0,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,BIODIESEL EXTRA,8400,1,05/02/2018 12:00:00 AM,2018,5,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia","(""4° 15' 0.00000'' N"", ""74° 10' 59.98800'' W"")",4.25,4.25
1,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,GASOLINA CORRIENTE OXIGENADA,9095,1,05/02/2018 12:00:00 AM,2018,5,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia","(""4° 15' 0.00000'' N"", ""74° 10' 59.98800'' W"")",4.25,4.25


In [129]:
central_location = 'Cra. 58d #146-51, Bogotá, Colombia'
central_location = central_location.replace('#','No')
API_KEY = '4901aad006d0464594531f2f481f52f6' # Colocar en otro sitio
R = GetLatLon2__opencage(central_location,API_KEY)
R

['Cra. 58d No146-51, Bogotá, Colombia', 4.60971, -74.08175]

In [130]:
geo_source = R[1],R[2]
geo_source

(4.60971, -74.08175)

In [131]:
radio = 1
unit = 'Km'

In [132]:
df_city =  transform_df_map(df_city)
df_city.head(2)

/tmp/ipykernel_9792/295678141.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['POINT'] = coordenadas


,Cod.Departamento,Departamento,Cod.Municipio,Municipio,Agente,Bandera,Direccion,Producto,Precio,Estado,Registro,Periodo,Mes,Direccion_2,Full_Address,Coords,Latitud,Longitud,POINT
0,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,BIODIESEL EXTRA,8400,1,05/02/2018 12:00:00 AM,2018,5,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia","(""4° 15' 0.00000'' N"", ""74° 10' 59.98800'' W"")",4.25,4.25,"(4.25, 4.25)"
1,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,GASOLINA CORRIENTE OXIGENADA,9095,1,05/02/2018 12:00:00 AM,2018,5,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia","(""4° 15' 0.00000'' N"", ""74° 10' 59.98800'' W"")",4.25,4.25,"(4.25, 4.25)"


In [133]:
df_city['POINT'][0]

(4.25, 4.25)

In [134]:
results = distance_estac(geo_source,df_city,radio,unit)
#results.reset_index(drop=True,inplace =  True)
#results.drop(columns= 'index' , inplace = True)

100%|████████████████████████████████████| 1278/1278 [00:00<00:00, 30959.99it/s]


In [136]:
results.head(2)

,Cod.Departamento,Departamento,Cod.Municipio,Municipio,Agente,Bandera,Direccion,Producto,Precio,Estado,...,Periodo,Mes,Direccion_2,Full_Address,Coords,Latitud,Longitud,POINT,SOURCE,DISTANCE
0,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,BIODIESEL EXTRA,8400,1,...,2018,5,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia","(""4° 15' 0.00000'' N"", ""74° 10' 59.98800'' W"")",4.25,4.25,"(4.25, 4.25)","(4.60971, -74.08175)",8679.21
766,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO UNION SAN FRANCISCO,ESSO,CARRERA 19D No. 64-80 SUR,BIOACEM AL 9%,8416,1,...,2018,5,CARRERA 19D No. 64-80 SUR,"CARRERA 19D No. 64-80 SUR, Bogota d.c., Colombia","(""4° 15' 0.00000'' N"", ""74° 10' 59.98800'' W"")",4.25,4.25,"(4.25, 4.25)","(4.60971, -74.08175)",8679.21


In [137]:
results['Agente'].unique()

array(['ESTACION DE SERVICIO METROPOLITANA',
       'ESTACION DE SERVICIO UNION SAN FRANCISCO',
       'ESTACION DE SERVICIO TERPEL AVENIDA DE CALI',
       'ESTACION DE SERVICIO BRIO ROCAMAR', 'ESTACION CLL 122 CON 19',
       'ESTACION DE SERVICIO VILLA ALSACIA',
       'ESTACION DE SERVICIO HAYUELOS CON AVENIDA CIUDAD DE CALI',
       'EDS TERPEL VILLA CLAUDIA', 'ESTACION DE SERVICIO NQS 8',
       'ESTACION DE SERVICIO CALLE 153', 'EDS ESSO EL TRIANGULO',
       'ESTACION DE SERVICIO PETROBRAS CANDELARIA',
       'ESTACION DE SERVICIO COUNTRY CLUB DE BOGOTA', 'EDS RESTREPO',
       'ESTACION DE SERVICIO AUTOMOTRIZ EXITO VILLA MAYOR',
       'ESTACION PETROBRAS PATIO BONITO',
       'ESTACION DE SERVICIO PRIVADA TESCOTUR SA',
       'ESTACIÓN DE SERVICIO AMERICAS 2',
       'ESTACION DE SERVICIO PRADERA DE SUBA',
       'ESTACION DE SERVICIO BOSA',
       'ESTACION DE SERVICIO AUTOMOTRIZ EXITO AMERICAS',
       'ESTACION DE SERVICIO TEXACO CIUDAD BOLIVAR',
       'ESTACION DE SERVIC

In [138]:
results['Producto'].unique()

array(['BIODIESEL EXTRA', 'BIOACEM AL 9%', 'GASOLINA CORRIENTE OXIGENADA',
       'GASOLINA EXTRA OXIGENADA', 'GASOLINA CORRIENTE', 'GASOLINA EXTRA'],
      dtype=object)

In [139]:
type(results)

pandas.core.frame.DataFrame

In [140]:
gdf_results = GeoDataFrame(results,
                           geometry =  points_from_xy(results.Longitud,results.Latitud))

In [141]:
type(gdf_results)

geopandas.geodataframe.GeoDataFrame

In [142]:
gdf_results.head(2)

,Cod.Departamento,Departamento,Cod.Municipio,Municipio,Agente,Bandera,Direccion,Producto,Precio,Estado,...,Mes,Direccion_2,Full_Address,Coords,Latitud,Longitud,POINT,SOURCE,DISTANCE,geometry
0,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO METROPOLITANA,MOBIL,CARRERA 120 No. 17-37,BIODIESEL EXTRA,8400,1,...,5,CARRERA 120 No. 17-37,"CARRERA 120 No. 17-37, Bogota d.c., Colombia","(""4° 15' 0.00000'' N"", ""74° 10' 59.98800'' W"")",4.25,4.25,"(4.25, 4.25)","(4.60971, -74.08175)",8679.21,POINT (4.25000 4.25000)
766,11,BOGOTA D.C.,11001,BOGOTA D.C.,ESTACION DE SERVICIO UNION SAN FRANCISCO,ESSO,CARRERA 19D No. 64-80 SUR,BIOACEM AL 9%,8416,1,...,5,CARRERA 19D No. 64-80 SUR,"CARRERA 19D No. 64-80 SUR, Bogota d.c., Colombia","(""4° 15' 0.00000'' N"", ""74° 10' 59.98800'' W"")",4.25,4.25,"(4.25, 4.25)","(4.60971, -74.08175)",8679.21,POINT (4.25000 4.25000)


In [143]:
gdf_results['geometry']

0      POINT (4.25000 4.25000)
766    POINT (4.25000 4.25000)
767    POINT (4.25000 4.25000)
768    POINT (4.25000 4.25000)
769    POINT (4.25000 4.25000)
                ...           
233    POINT (4.79803 4.79803)
743    POINT (4.82691 4.82691)
742    POINT (4.82691 4.82691)
744    POINT (4.82691 4.82691)
745    POINT (4.82691 4.82691)
Name: geometry, Length: 1278, dtype: geometry

In [144]:
oil = list(gdf_results['Producto'].unique())
oil

['BIODIESEL EXTRA',
 'BIOACEM AL 9%',
 'GASOLINA CORRIENTE OXIGENADA',
 'GASOLINA EXTRA OXIGENADA',
 'GASOLINA CORRIENTE',
 'GASOLINA EXTRA']

In [145]:
gdf_results_2 = gdf_results[gdf_results['Producto']==oil[1]]
gdf_results_2.reset_index(drop=True,inplace =  True)
#gdf_results_2.drop(columns= 'index' , inplace = True)

In [146]:
gdf_results_2.columns

Index(['Cod.Departamento', 'Departamento', 'Cod.Municipio', 'Municipio',
       'Agente', 'Bandera', 'Direccion', 'Producto', 'Precio', 'Estado',
       'Registro', 'Periodo', 'Mes', 'Direccion_2', 'Full_Address', 'Coords',
       'Latitud', 'Longitud', 'POINT', 'SOURCE', 'DISTANCE', 'geometry'],
      dtype='object')

In [147]:
gdf_results_2['Precio'].min()

7337

In [148]:
gdf_results_2['Precio'].max()

9260

In [149]:
def marker_rest(df,mapa,unit,oil,icono):
    
    df = df[df['Producto']==oil]
    df = df.reset_index()
    df = df.drop(columns = 'index')
    
    for i in range(len(df)):
        
        if df['Precio'][i]==df['Precio'].min():
        
            html =  f"""<b>MARCA:</b> {df.Bandera[i]} <br>
                    <b>NAME:</b> {df.Agente[i]} <br>
                    <b>PRODUCTO:</b> {df.Producto[i]} <br>
                    <b>PRECIO:</b> {df.Precio[i]} <br>
                    <b>DISTANCE:</b> {round(df.DISTANCE[i],2)}<br>
                    <b>DIRECCION:</b> {df.Direccion[i]}<br>
                    <b>UNIT:</b> {unit}<br>"""
            iframe = folium.IFrame(html,figsize=(6, 3))
            popup = folium.Popup(iframe)
            



            folium.Marker(location=[float(df['Latitud'][i]),float(df['Longitud'][i])],
                               icon=folium.Icon(color='darkgreen', icon_color='white',
                               icon=icono, prefix='glyphicon'),
                               popup = popup).add_to(mapa)
        
        elif df['Precio'][i]==df['Precio'].max():
        
            html =  f"""<b>MARCA:</b> {df.Bandera[i]} <br>
                    <b>NAME:</b> {df.Agente[i]} <br>
                    <b>PRODUCTO:</b> {df.Producto[i]} <br>
                    <b>PRECIO:</b> {df.Precio[i]} <br>
                    <b>DISTANCE:</b> {round(df.DISTANCE[i],2)}<br>
                    <b>DIRECCION:</b> {df.Direccion[i]}<br>
                    <b>UNIT:</b> {unit}<br>"""
            iframe = folium.IFrame(html,figsize=(6, 3))
            popup = folium.Popup(iframe)



            folium.Marker(location=[float(df['Latitud'][i]),float(df['Longitud'][i])],
                               icon=folium.Icon(color='darkred', icon_color='white',
                               icon=icono, prefix='glyphicon'),
                               popup =popup).add_to(mapa)
        else :
            html =  f"""<b>MARCA:</b> {df.Bandera[i]} <br>
                    <b>NAME:</b> {df.Agente[i]} <br>
                    <b>PRODUCTO:</b> {df.Producto[i]} <br>
                    <b>PRECIO:</b> {df.Precio[i]} <br>
                    <b>DISTANCE:</b> {round(df.DISTANCE[i],2)}<br>
                    <b>DIRECCION:</b> {df.Direccion[i]}<br>
                    <b>UNIT:</b> {unit}<br>"""
            iframe = folium.IFrame(html,figsize=(6, 3))
            popup = folium.Popup(iframe)



            folium.Marker(location=[float(df['Latitud'][i]),float(df['Longitud'][i])],
                               icon=folium.Icon(color='orange', icon_color='white',
                               icon=icono, prefix='glyphicon'),
                               popup =popup).add_to(mapa)
                           
    return

# Show Map

In [151]:
m = folium.Map([geo_source[0],geo_source[1]], zoom_start= 15)

folium.Circle(
    
    radius = int(radio)*1000,
    location = [geo_source[0],geo_source[1]],
    color = 'green',
    fill = 'green'

).add_to(m)

folium.Marker(

            location = [geo_source[0],geo_source[1]],
            icon =  folium.Icon(color='black',icon_color='white',
            icon = "home",prefix = 'glyphicon'),
            popup = "<b>Centroid</b>").add_to(m)

marker_rest(gdf_results_2,m,unit,oil[1],'usd')

m